In [3]:
    import CommonMark
    from CommonMark import Parser
    from CommonMark.render.renderer import Renderer
    from textwrap import indent, dedent
    from importnb import Notebook
    from collections import UserList
    from abc import abstractmethod, ABCMeta
    try:
        from IPython.display import display, Markdown, HTML
        from IPython.core.inputtransformer import InputTransformer
        from IPython import get_ipython
    except:
        class InputTransformer:
            def __init__(self, *args, **kwargs): ...
            def transform_cell(self, str): return dedent(str)
            
        def get_ipython(): ...

## Markdown to code

Markdown is converted to code by persisting all code_blocks and replacing all non-code lines with blank lines.  The blank line replacement assures the right line numbering.

* Create a custom CommonMark renderer for code only.  It maintains the lines and positions of the original source to give better error messages.

* The commonmark renderer only catches code cells

    > There is an interesting discussion to be had about the role of inline cells.

In [5]:
    class CodeRenderer(Renderer):
        def code(self, node, entering):
            if self.user_expressions is not None: self.user_expressions.update({node.literal:node.literal})
        def code_block(self, node, entering):
            while len(self.buf.splitlines()) <  node.sourcepos[0][0]-1:
                self.out("\n")
            self.out(indent(node.literal, ' '*(
                node.sourcepos[0][1])))
            
        repr = staticmethod(Markdown)
            
        def __call__(self, str, user_expressions=None):
            self.user_expressions = user_expressions
            return dedent(self.render(Parser().parse(str)))

In [13]:
    class CallableTransformer(InputTransformer, UserList, metaclass=ABCMeta):
        """Define a __call__ method define a callable transformer."""
        
        push = UserList.append

        def reset(self, str = ""):
            while self.data: str += self.data.pop(0) + '\n'
            return self(str)
        
        @abstractmethod
        def __call__(self, str): raise NotImplemented()
            
        

In [14]:
    renderer = CodeRenderer()

In [15]:
    class MarkdownTransformer(CallableTransformer):
        def __call__(self, str):  
            str = renderer(str)
            if str.strip(): return str
            return ''

In [16]:
    class MarkdownImporter(Notebook):
        extensions = '.md.ipynb',
        def format(self, str): return super().format(renderer(str))

In [17]:
    try:
        from IPython.display import HTML, Javascript
        
        style = HTML("""<style>
        .code-cell-toggle {
            position: fixed;
            bottom: 20px;
            right: 20px;
        }
        .code-cell-hidden .code_cell .input {
            display: none;
        }
        </style>""")

        toggle = Javascript("""$("button.code-cell-toggle").remove();
        var btn = $("<button/>", {"class": "code-cell-toggle"}).text("</>");
        btn.bind("click", function(){
            $("body").toggleClass("code-cell-hidden");
        });
        $("#notebook").append(btn);
        if (!window.location.host.startsWith('localhost')){
            $("body").addClass("code-cell-hidden");
        };""")
    except: ...

In [18]:
    def load_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager.physical_line_transforms = [
            MarkdownTransformer()] + [
            object for object in ip.input_transformer_manager.physical_line_transforms
            if not isinstance(object, MarkdownTransformer)]
        try:
            from IPython.display import display
            display(style, toggle)
        except: ...
            
    def unload_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager.physical_line_transforms = list(
            object for object in ip.input_transformer_manager.physical_line_transforms
            if not isinstance(object, MarkdownTransformer)
        )

In [19]:
    class Test(__import__('unittest').TestCase): 
        def setUp(Test):
            %reload_ext pidgin
            %reload_ext pidgin.markdown
            load_ipython_extension()
            from nbformat import write, v4
            with open('test_markdown.md.ipynb', 'w') as file:
                write(v4.new_notebook(cells=[v4.new_code_cell("""Some paragraph\n\n    a=42""")]), file)
                
        def runTest(Test):
            global test_markdown
            import test_markdown
            assert test_markdown.__file__.endswith('.ipynb')
            assert test_markdown.a is 42
            
        def tearDown(Test):
            %unload_ext pidgin.markdown
            from pathlib import Path
            #__import__('os').remove('test_markdown.md.ipynb')
            unload_ipython_extension()

In [20]:
    if __name__ == '__main__':
        %reload_ext pidgin
        load_ipython_extension()

<IPython.core.display.Javascript object>

### if __name__ == '__main__': 
        __import__('unittest').TextTestRunner().run(Test())